# Databases - part II
## Data Modeling
The process of designing & planning a database structure.

[**Normalisation**](https://www.geeksforgeeks.org/introduction-of-database-normalization/)<br>
The primary objective for normalizing the relations is to eliminate the below anomalies. Failure to reduce anomalies results in data redundancy, which may threaten data integrity and cause additional issues as the database increases. Normalization consists of a set of procedures that assist you in developing an effective database structure.
* *Insertion Anomalies*: Insertion anomalies occur when it is not possible to insert data into a database because the required fields are missing or because the data is incomplete.
    * E.g. if a database requires that every record has a primary key, but no value is provided for a particular record, it cannot be inserted into the database.
* *Deletion anomalies*: Deletion anomalies occur when deleting a record from a database and can result in the unintentional loss of data.
    * E.g. if a database contains information about customers and orders, deleting a customer record may also delete all the orders associated with that customer.
* *Updation anomalies*: Updation anomalies occur when modifying data in a database and can result in inconsistencies or errors. 
    * E.g. if a database contains information about employees and their salaries, updating an employee’s salary in one record but not in all related records could lead to incorrect calculations and reporting.

### Princess Knights Dragons
Develop the best data model you can (in the form of an ERD - no code allowed) to track the dragon-knight-princess ecosystem
These interactions are only superficially so simple:
* Sometimes knights fail
* Sometimes a dragon will abduct more than one princess (sequentially or concurrently)
* Sometimes a dragon is a troll, or a knight a prince, or a princess a goose girl
* Sometimes a princess will do the rescuing (shame on you for not even considering that)
* A princess (or knight, or dragon) might be involved in such interactions multiple times
* Abductions imply a change in location
 

## Manipulating Databases
pjt:    SQL_1
pw:     cygcan-wywgu4-qYsqyj



We create a `flavour` table:
```sql
CREATE TABLE flavour (
  flavour_id INT GENERATED ALWAYS AS identity,
  flavour_name VARCHAR(30) NOT NULL,
  vegan boolean default false,
  primary key(flavour_id)
);
```



To populate our data we do the following:
```sql
INSERT INTO flavour (
  flavour_name,
  vegan
) VALUES (
  'strawberry',
 TRUE
);
```



To populate more than one row into the table:
```sql
INSERT INTO flavour (flavour_name)
VALUES 
  ('chocolate'),
  ('toffee'),
  ('pistachio'),
  ('rocky_road'),
  ('cardamon'),
  ('salted caramel'),
  ('salmon');
```



To delete a row:
```sql
DELETE FROM flavour
WHERE flavour_name = 'salmon';
```


To update a row
```sql
UPDATE flavour
SET vegan = true
WHERE flavour_name = 'pistachio'
RETURNING *;
```


Create a `customer_order` table
```sql
CREATE TABLE customer_order (
  order_id INT GENERATED ALWAYS AS IDENTITY,
  customer_name VARCHAR(50) NOT NULL,
  flavour_id INT NOT NULL,
  scoops INT DEFAULT 1,
  PRIMARY KEY (order_id),
  FOREIGN KEY (flavour_id) REFERENCES flavour(flavour_id)
);
```


Insert a customer into the customer_table
```sql
INSERT INTO customer_order (customer_name, flavour_id, scoops)
VALUES ('Nancy', 1, 3);
```



We get an error with the below because we do not have `flavour_id 36`
```sql
INSERT INTO  customer_order (customer_name, flavour_id, scoops)
VALUES ('Max', 36, 2)
```



We correct this with the below
```sql
INSERT INTO  customer_order (customer_name, flavour_id, scoops)
VALUES ('Max', 6, 2);
```



To see what the flavour the customer ordered, we use the below SELECT and JOIN
```sql
SELECT C.customer_name, F.flavour_name
FROM customer_order AS C
LEFT JOIN flavour AS F
ON (C.flavour_id = F.flavour_id)
```


## Connecting to Databases


## Security & Auth Concepts


## Implementing Auth


## Extending Auth